## 在语义化搜索中使用嵌入向量

我们可以通过嵌入搜索查询，然后找到最相似的评论，以非常高效且低成本的方式对所有评论进行语义搜索。

In [1]:
%pip install pandas
%pip install numpy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

datafile_path = "./archive/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)


在这里，我们比较查询和文档嵌入的余弦相似度，并展示前 top_n 个最佳匹配项。

In [3]:
# 导入获取嵌入向量和计算余弦相似度的函数
from utils.embeddings_utils import get_embedding, cosine_similarity

def search_reviews(df, product_description, n=3, pprint=True):
    """
    根据产品描述，在评论数据集中查找最相似的评论。
    
    参数:
    df: pandas.DataFrame, 包含评论文本和对应嵌入向量的数据集。
    product_description: str, 产品的描述，用于生成产品嵌入向量。
    n: int, 默认为3，指定返回的最相似评论的数量。
    pprint: bool, 默认为True，控制是否打印最相似的评论。
    
    返回:
    pandas.Series, 包含最相似评论的标题和内容的字符串。
    """
    # 为产品描述生成嵌入向量
    product_embedding = get_embedding(
        product_description,
        model="text-embedding-3-small"
    )
    
    # 计算每条评论的嵌入向量与产品描述嵌入向量的余弦相似度
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))
    
    # 根据相似度排序，并提取最相似的n条评论的标题和内容
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )
    
    # 如果指定，打印最相似的评论
    if pprint:
        for r in results:
            print(r[:200])
            print()
    
    return results

# 调用函数，搜索与"delicious beans"描述最相似的3条评论
results = search_reviews(df, "delicious beans", n=3)


Delicious!:  I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning

Fantastic Instant Refried beans:  Fantastic Instant Refried Beans have been a staple for my family now for nearly 20 years.  All 7 of us love it and my grown kids are passing on the tradition.

Delicious:  While there may be better coffee beans available, this is my first purchase and my first time grinding my own beans.  I read several reviews before purchasing this brand, and am extremely 



# 来个形象点的例子
咱们的嵌入向量数据都是英文的，但是我输入“猫粮”，仍然能查到和猫粮相关的评论，这就是语义化搜索！

In [ ]:
results = search_reviews(df, "猫粮", n=3)


又乱又明显不好吃：我的猫不是很喜欢。当然，它会舔掉肉汁，但把那些像肉的小块留得像骨头一样干。感觉像是在省钱，但如果你的猫在吃就不是了。 

又乱又明显不好吃：我的猫不是很喜欢。当然，它会舔掉肉汁，但把那些像肉的小块留得干巴巴的。感觉像是在省钱，但如果你的猫在吃就不是了。 

这些猫喜欢它：我的 7 只猫喜欢这种食物，但对人类来说有点难吃。一块块鲭鱼在深色的肉汤里游动。它被宣传为“完整”的食物，包含胡萝卜、豌豆和意大利面。 